In [ ]:
import sys
import numpy as np
import numpy.linalg as la
import sympy as sp
from typing import Tuple

In [ ]:
# SVD

# Pseudo-inverse:
def pseudo_inv_Sigma(Sigma:np.ndarray)->np.ndarray:
  m,n = Sigma.shape
  k = min(m,n)
  Sigma_plus = np.zeros(Sigma.shape)
  for i in range(k):
    sig_i = Sigma[i][i]
    if(Sigma[i][i]!=0): Sigma_plus[i][i] = 1/sig_i
  return Sigma_plus

def inv_A(U:np.ndarray,Sigma:np.ndarray,V:np.ndarray)->np.ndarray:
  if(la.cond(Sigma)<1/sys.float_info.epsilon):
    A_inv = V @ la.inv(Sigma) @ U.T
    message = "\n====== Normal Inverse of A =====\n"
  else:
    A_inv = V @ pseudo_inv_Sigma(Sigma) @ U.T
    message = "\n====== Pseudo Inverse of A =====\n"
  print(message)
  return A_inv

# Get SVD from (A^T)(A)
# Returns (Sigma, V)
def ATA_to_SVD(ATA:np.ndarray)->tuple:
  V = np.zeros(ATA.shape)
  Sigma = np.zeros(ATA.shape)
  vals, vecs = la.eig(ATA)
  orders = vals.argsort()
  for i in range(len(vals)):
    order = orders[i]
    val = vals[order]
    vec = vecs[:,order]
    V[:,i] = vec
    Sigma[i][i] = np.sqrt(val)
  return Sigma, V

# Get SVD from (A)(A^T)
# Returns (U, Sigma)
# AAT = XDX^-1, X is eigvecs of AAT, D is eigvals
def AAT_get_U_Sigma(X,D):
  U = X
  Sigma = np.sqrt(D)
  return (U,Sigma)

# Get SVD from XDX factorizations (or eigenpairs)
# Returns (U, Sigma, V)
def Eig_to_SVD(A,X,D,AV=None):
  Sigma = np.sqrt(D)
  V = np.zeros(X.shape)
  for i in range(len(X[0])):
    V[:,i] = X[:,i]
  if(AV is not None): 
    U = AV @ (la.inv(Sigma))
  else: 
    U = A @ V @ (la.inv(Sigma))
  return U, Sigma, V


# get condition number of A by using sigma
def get_cond(Sigma: np.ndarray)->float:
  d = np.diag(Sigma)
  sig_max = np.amax(d)
  sig_min = np.amin(d)

  if(sig_min==0): return np.inf
  return sig_max/sig_min

def get_norm(Sigma: np.ndarray):
  return np.max(Sigma)

# get Norm(A^-1)
def norm_inv(Sigma:np.ndarray):
  d = np.diag(Sigma)
  sig_r = np.amin(d)
  if(sig_r==0): return 0
  return 1/sig_r

# Time Complexity of SVD
# this function will calculate the cost for the new matrix
# the matrix with given time is n0 x m0, time consumed is t0
# the new given matrix is n1 x m1
def get_time(n0,m0,t0,n1,m1):
  # SVD = alpha(mn^2+n^3)
  frac = (n1**3+m1*(n1**2))/(n0**3+m0*(n0**2))
  return frac*t0

# get all info of A
def get_all_info(U,Sigma,read_info=0):
  n = np.min(Sigma.shape)
  print(n)
  U_r = U[:,:n]
  Sigma_r = Sigma[:n,:n]
  cond = get_cond(Sigma)
  norm = get_norm(Sigma)
  if(read_info == 0):
    print("2-Norm of A is        : "+str(norm))
    print("2-Norm Cond(A) is     : "+str(cond))
    print("Shape of U reduced    : "+str(U_r.shape))
    print("Shape of Sigma reduced: "+str(Sigma_r.shape))
    print("U reduced             : "+str(U_r))
    print("Sigma reduced         : "+str(Sigma_r))
  return U_r,Sigma_r

# A_approximation
# Image compression
def A_approx(U, Sigma, VT, rank):
  for i in range(rank):
    ui = U[:,[i]]
    vti = VT[[i],:]
    sig = Sigma[i][i]
    if(i==0): approx = sig*(ui @ vti)
    else: approx += sig*(ui @ vti)
  norm_app = Sigma[rank][rank]
  norm_res = norm_app/Sigma[0][0]
  print("A_rank        : "+str(approx.tolist()))
  print("Norm(A-A_rank): "+str(norm_app))
  print("Norm(Residual): "+str(norm_res)) 
  return approx


  

In [ ]:
## Least Square

# Get x
def get_x_SVD(U: np.ndarray,Sigma: np.ndarray,VT: np.ndarray,b: np.ndarray):
  n = min(Sigma.shape)
  Sigma_pr = pseudo_inv_Sigma(Sigma)[:n,:n]
  U_r = U[:,:n]
  x = VT.T @ Sigma_pr @ U_r.T @ b
  return x

# Get Residual: ||b-Ax||_2
def get_residual(U: np.ndarray,Sigma: np.ndarray,VT: np.ndarray,b: np.ndarray):
  x = get_x_SVD(U, Sigma, VT, b)
  Ax = U @ Sigma @ VT @ x
  out = la.norm(b-Ax,2)
  return out

# get Φ(x)
def function_residual(f,x,x0,y0):
  r = []
  for i in range(len(x0)):
    yi = f
    xi = x0[i]
    if(type(xi)==list):
      for j in range(len(x)):
        yi = yi.subs(x[j],xi[j])
    else:
      yi = yi.subs(x[0],xi)
    r.append(abs(y0[i] - yi))
  r = np.array(r,dtype="float64")
  return f,la.norm(r,2)

# get A in least square
def construct_A(ts,t,t0):
  n = ts.shape[0]
  m = len(t0)
  A = np.zeros((m,n))
  for i in range(m):
    for j in range(n):
      if(type(ts[j])!=int and type(ts[j])!=float):
        A[i][j] = ts[j].subs(t[0],t0[i])
      else:
        A[i][j] = ts[j]
  return A

def get_x_least_square(A, b):
  result = la.lstsq(A, b,rcond=-1)
  x = result[0]
  residual = result[1]
  return x, residual

def get_A_rank(x0):
  print("如果x项数>"+str(len(set(x0.flatten()))) + ", Rank = "+str(len(set(x0.flatten()))))
  print("如果x项数<="+str(len(set(x0.flatten()))) + ", Rank = x项数")

def get_Normal(ts,t,t0,y0):
  A = construct_A(ts,t,t0)
  M = A.T @ A
  n = A.T @ y0.T
  print("Normal: Mc=n")
  print("M = AT @ A")
  print("n = AT @ b")
  print("M:"+str(M))
  print("n:"+str(n))
  return M, n

In [ ]:
Sigma = np.array([[8,0],[0,3],[0,0]])
U = np.array([[-0.70,0.58,0.42],[-0.67,-0.33,-0.67],[-0.25,-0.75,0.61]])
get_all_info(U,Sigma)

In [ ]:
U = np.array([[0.00, 0.92, 0.29, -0.26], [-0.25, 0.06, 0.53, 0.81], [-0.80, -0.23, 0.33, -0.45], [-0.55, 0.30, -0.73, 0.28]])
sigma = np.array([[8.00, 0.00, 0.00], [0.00, 6.00, 0.00], [0.00, 0.00, 5.00], [0.00, 0.00, 0.00]])
Vt = np.array([[-0.68, -0.65, -0.33], [0.47, -0.05, -0.88], [0.56, -0.75, 0.34]])
A_approx(U, sigma, Vt, 2)

In [ ]:
U=np.array([[0,1/np.sqrt(2),1/np.sqrt(2)],[0,1/np.sqrt(2),-1/np.sqrt(2)],[1,0,0]])
sigma = np.array([[5,0,0],[0,3,0],[0,0,2]])
V = np.array([[1,0,0],[0,0,1],[0,1,0]])
b = np.array([[np.sqrt(2)],[np.sqrt(2)],[1]])

U_r, sigma_r = get_all_info(U,sigma, read_info=1)
sigma_p = pseudo_inv_Sigma(sigma_r)
V @ sigma_p @ U_r.T @ b

In [ ]:
U = np.array([[1,0,0],[0,-1/np.sqrt(2),-1/np.sqrt(2)],[0,1/np.sqrt(2),1/np.sqrt(2)]])
Sigma = np.array([[6,0,0],[0,6,0],[0,0,0]])
VT = np.array([[-1*np.sqrt(3)/2,1/2,0],[-1/2,-1*np.sqrt(3)/2,0],[0,0,1]])
b = np.array([[6],[0],[0]])

get_x_SVD(U,Sigma,VT,b)

In [ ]:
U = np.array([[1/np.sqrt(2),-1/np.sqrt(2),0,0],[1/np.sqrt(2),1/np.sqrt(2),0,0],[0,0,0,1],[0,0,1,0]])
Sigma = np.array([[4,0,0],[0,10,0],[0,0,0],[0,0,0]])
VT = np.array([[1,0,0],[0,1,0],[0,0,1]])
b = np.array([[6],[8],[2],[2]])
get_residual(U,Sigma, VT,b)

2.8284271247461903

In [ ]:
x = sp.symbols("x:1")
fs = np.array([1.5*x[0]+7,4.5*x[0]+2,3*x[0]+3,5*x[0]+1])
x0 = [1,2,5]
y0 = [2,18,12]
for f in fs:
  print(function_residual(f,x,x0,y0))



(1.5*x0 + 7, 10.606601717798213)
(4.5*x0 + 2, 15.016657417681207)
(3*x0 + 3, 11.532562594670797)
(5*x0 + 1, 16.15549442140351)


In [ ]:
t = sp.symbols("t:1")
t0 = [0.5,1.0,1.5,2.0,2.5,3.0,3.5]
ts = np.array([1,sp.sin(t[0]*sp.pi),sp.sin(t[0]*sp.pi/2),sp.sin(t[0]*sp.pi/4)])
construct_A(ts,t,t0)

array([[ 1.        ,  1.        ,  0.70710678,  0.38268343],
       [ 1.        ,  0.        ,  1.        ,  0.70710678],
       [ 1.        , -1.        ,  0.70710678,  0.92387953],
       [ 1.        ,  0.        ,  0.        ,  1.        ],
       [ 1.        ,  1.        , -0.70710678,  0.92387953],
       [ 1.        ,  0.        , -1.        ,  0.70710678],
       [ 1.        , -1.        , -0.70710678,  0.38268343]])

In [ ]:
# Least-Squares Minimum Norm Solution
a1 = np.array([[0,0,1,0],[1,0,0,0],[0,0,0,1],[0,0,0,0],[0,1,0,0]])
a2 = np.array([[6,0,0,0],[0,2,0,0],[0,0,1,0],[0,0,0,0]])
a3 = np.array([[0.00,0.00,-0.05,1.00],[0.00,0.00,-1.00,-0.05],[0.92,-0.39,0.00,0.00],[-0.39,-0.92,0.00,0.00]])
A = a1 @ a2 @ a3
b = np.array([3,9,9,2,6])
print(get_x_least_square(A,b.T))

(array([ 2.76414622, -1.17175764, -3.06733167,  1.34663342]), array([], dtype=float64))


In [ ]:
# Getting the rank of matrix A from data points
x0 = np.array([9.5,11.0,7.5,8.0,6.5,6.0,7.5,9.5,6.0])
get_A_rank(x0)

如果x项数>6, Rank = 6
如果x项数<=6, Rank = x项数


In [ ]:
# Obtain the SVD from eigendecomposition
X = np.array([[-0.99,-0.16],[-0.16,0.99]])
D = np.array([[10,0],[0,3]])
AAT_get_U(X,D)

(array([[-0.99, -0.16],
        [-0.16,  0.99]]), array([[3.16227766, 0.        ],
        [0.        , 1.73205081]]))

In [ ]:
# Find RHS of normal equation
t0 = np.array([1,4,6])
y0 = np.array([-5,2,6])
t = sp.symbols("t:1")
ts = np.array([sp.cos(t[0]),sp.sqrt(t[0])])

get_Normal(ts,t,t0)

Normal: Mc=n
M = AT @ A
n = AT @ b
M:[[ 1.64110354  1.58494233]
 [ 1.58494233 11.        ]]
n:[ 1.75222295 13.69693846]


(array([[ 1.64110354,  1.58494233],
        [ 1.58494233, 11.        ]]), array([ 1.75222295, 13.69693846]))

In [ ]:
# Find LHS of normal equation
t0 = np.array([1,3,6])
y0 = np.array([0.74,0.79,1.0])
t = sp.symbols("t:1")
ts = np.array([sp.sin(t[0]),sp.sqrt(t[0])])

A = construct_A(ts,t,t0)
M = A.T @ A
print(M)

[[ 0.8060613   0.40147261]
 [ 0.40147261 10.        ]]


In [ ]:
# Exam Score vs. Study Time
import numpy as np
import numpy.linalg as la

def get_A(b):
  n = 4
  m = b.shape[0]
  A = np.zeros((m,n))
  for i in range(m):
    x = b[i]
    A[i] = [(x+2)**(2.5), np.exp(x+1),x,1]
  return A
 
A = get_A(study_time)
coeffs = la.lstsq(A,scores, rcond=-1)[0]

In [ ]:
# Solve linear system using SVD
import numpy as np
bT = b.T
inv_sigma = sigma**(-1)
M1 = right_multiply_with_U(bT)
M2 = np.diag(inv_sigma).T
W = (M1@M2).T
x = left_multiply_with_V(W)

In [ ]:
# Estimate Sales
import numpy as np
import numpy.linalg as la
def get_A(years):
    n = years.shape[0]
    A = np.zeros((n,2))
    for i in range(years.shape[0]):
        A[i][0] = 1
        A[i][1] = years[i]
    return A
    
A = get_A(given_year)
coef = la.lstsq(A,given_sales,rcond=-1)[0]

pred_sales = np.zeros(pred_year.shape)
for i in range(pred_year.shape[0]):
    pred_sales[i] = coef[0]+coef[1]*pred_year[i]

In [ ]:
# Miles Driven vs. Gas Money
import numpy as np
import numpy.linalg as la
n = miles_driven.shape[0]
A = np.zeros((n,4))

for i in range(n):
    x = miles_driven[i]
    A[i] = [1, x, (2*(x**2)-1),(4*(x**3)-3*x)]

coeffs = la.lstsq(A,gas_money,rcond = -1)[0]

In [ ]:
# Modeling Rabbit Population
import numpy as np
import numpy.linalg as la
# ln(fx) = ln(c0)+ln(exp(c1x))=ln(c0)+c1*x
population = np.log(population)
n = population.shape[0]
A = np.zeros((n,2))

for i in range(n):
    x = i
    A[i] = [1, x]

coeffs = la.lstsq(A,population,rcond = -1)[0]
c0 = np.exp(coeffs[0])
c1 = coeffs[1]

In [ ]:
# Find LHS of normal equation
t0 = np.array([1,4,5])
y0 = np.array([0.74,0.79,1.0])
t = sp.symbols("t:1")
ts = np.array([sp.sqrt(t[0]), sp.sin(t[0])])
get_Normal(ts,t,t0)

Normal: Mc=n
M = AT @ A
n = AT @ b
M:[[10.         -2.81635387]
 [-2.81635387  2.2003592 ]]
n:[ 4.55606798 -0.93410972]


(array([[10.        , -2.81635387],
        [-2.81635387,  2.2003592 ]]), array([ 4.55606798, -0.93410972]))

In [ ]:
U = np.array([[-1,0],[0,1]])
Sigma = np.array([[2,0],[0,0]])
VT = np.array([[np.sqrt(3)/2,1/2],[-1/2,np.sqrt(3)/2]])
b = np.array([9,0]).T

get_x_SVD(U,Sigma,VT,b)

array([-3.89711432, -2.25      ])

In [ ]:
t0 = np.array([1,5,7])
t = sp.symbols("t:1")
ts = np.array([sp.sin(t[0]), t[0]**2])
y0 = np.array([-2,2,4])
get_Normal(ts,t,t0,y0)

Normal: Mc=n
M = AT @ A
n = AT @ b
M:[[2.05924057e+00 9.06070746e+00]
 [9.06070746e+00 3.02700000e+03]]
n:[ -0.97284412 244.        ]


(array([[2.05924057e+00, 9.06070746e+00],
        [9.06070746e+00, 3.02700000e+03]]),
 array([ -0.97284412, 244.        ]))

In [ ]:
# Find the Residual
U = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]])
Sigma = np.array([[6,0,0],[0,8,0],[0,0,0],[0,0,0]])
VT = np.array([[1/np.sqrt(2),1/np.sqrt(2),0],[1/np.sqrt(2),-1/np.sqrt(2),0],[0,0,1]])
b = np.array([4,7,4,3]).T

get_residual(U,Sigma,VT,b)

5.0